In [18]:
import timeit


def longest_common_substring(s1, s2):
    maxLongest = 0
    offset = 0
    for i in range(0, len(s1)):
        longest = 0
        if ((i == len(s1) - len(s2) - 2)):
            break            
        for j in range(0, len(s2)):
            if (i+j < len(s1)):
                if s1[i+j] == s2[j]:
                    longest = longest + 1
                    if (maxLongest < longest):
                        maxLongest = longest
                        offset = i
                else:
                    break
            else:
                break
    return maxLongest, offset

    
def encode_lz78(text):
    dictionary = dict()
    i = 0
    index = 1
    encodedNumbers = []
    encodedLetters = []
    while i < len(text):
        stringToBeSaved = text[i]
        indexInDictionary = 0
        while stringToBeSaved in dictionary:
            indexInDictionary = dictionary[stringToBeSaved]
            if (i == len(text) - 1):
                stringToBeSaved = " "
                break
            i = i + 1
            stringToBeSaved = stringToBeSaved + text[i]
        encodedNumbers.append(indexInDictionary)
        encodedLetters.append(stringToBeSaved[len(stringToBeSaved) - 1])
        if (stringToBeSaved not in dictionary):
            dictionary[stringToBeSaved] = index
            index = index + 1
        i = i + 1

    return encodedNumbers, encodedLetters, dictionary
        
def decode_lz78(encodedNumbers, encodedLetters, dictionary):
    i = 0
    while i < len(encodedNumbers):
        if (encodedNumbers[i] != 0):    
            print(list(dictionary.keys())[list(dictionary.values()).index(encodedNumbers[i])], end="")
        print(encodedLetters[i], end="")
        i = i+1
    
def encode_lz77(text, searchWindowSize, previewWindowSize):
    encodedNumbers = []
    encodedSizes = []
    encodedLetters = []
    i = 0
    while i < len(text):
        if i < previewWindowSize:
            #print ("<{0}, {1}, {2}>".format(0, 0, text[i]))
            encodedNumbers.append(0)
            encodedSizes.append(0)
            encodedLetters.append(text[i])
            i = i + 1
        else:
            previewString = text[i:i+previewWindowSize]
            searchWindowOffset = 0
            if (i < searchWindowSize):
                searchWindowOffset = i
            else:
                searchWindowOffset = searchWindowSize
            searchString = text[i - searchWindowOffset:i] 
            result = longest_common_substring(searchString + previewString, previewString) # searchString + prevString, prevString   
            nextLetter = ''
            if (result[0] == len(previewString)):
                if (i + result[0] == len(text)):
                    nextLetter = ''
                else:
                    nextLetter = text[i+previewWindowSize]
            else:
                nextLetter = previewString[result[0]]
            if (result[0] == 0):
                #print ("<{0}, {1}, {2}>".format(0, 0, nextLetter))
                encodedNumbers.append(0)
                encodedSizes.append(0)
                encodedLetters.append(nextLetter)
            else:
                encodedNumbers.append(searchWindowOffset - result[1])
                encodedSizes.append(result[0])
                encodedLetters.append(nextLetter)
            i = i + result[0] + 1
    return encodedNumbers, encodedSizes, encodedLetters


def decode_lz77(encodedNumbers, encodedSizes, encodedLetters):
    i = 0
    decodedString = []
    while i < len(encodedNumbers):
        if (encodedNumbers[i] == 0): 
            decodedString.append(encodedLetters[i])   
        else:
            currentSize = len(decodedString)
            for j in range(0, encodedSizes[i]):
                decodedString.append(decodedString[currentSize-encodedNumbers[i]+j])
            decodedString.append(encodedLetters[i])   
        i = i+1
    return decodedString


def encode_lzw(text):
    dictionary = dict()
    i = 0
    index = 1
    while i < len(text):
        if text[i] in dictionary:
            i = i + 1
        else:
            dictionary[text[i]] = index
            index = index + 1

    i = 0
    encoded = []
    while i < len(text):
        j = 0
        stringToBeSaved = text[i]
        while stringToBeSaved in dictionary and i+j < len(text):
            indexInDictionary = dictionary[stringToBeSaved]
            length = len(stringToBeSaved)
            if (i+j == len(text) - 1):
                break
            j = j + 1
            stringToBeSaved = stringToBeSaved + text[i+j]
        i = i + length
        encoded.append(indexInDictionary)
        if (stringToBeSaved not in dictionary):
            dictionary[stringToBeSaved] = index
        index = index + 1

    return encoded, dictionary
 

def decode_lzw(encoded, dictionary):
    i = 0
    while i < len(encoded):
        print(list(dictionary.keys())[list(dictionary.values()).index(encoded[i])], end="")
        i = i+1

    
if __name__ == "__main__":
#     stringToEncode = "aaabbbbbbaaaabbbb"
    stringToEncode = "wszczebrzeszyniechrzaszczbrzmiwtrzcinie"
    
    searchWindowSize = 4
    previewWindowSize = 6
    start3 = timeit.default_timer()
    [encodedNumbers, encodedSizes, encodedLetters] = encode_lz77(stringToEncode, searchWindowSize, previewWindowSize)
    stop3 = (timeit.default_timer() - start3) * 1000
    print("Zakodowany ciąg w LZ77: ", end="")
    i = 0
    while i < len(encodedNumbers):
        print ("<{0}, {1}, {2}>".format(encodedNumbers[i], encodedSizes[i], encodedLetters[i]), end=" ")
        i = i + 1
    print("\nCzas kodowania LZ77: ", stop3)
    start4 = timeit.default_timer()
    decodedString = decode_lz77(encodedNumbers, encodedSizes, encodedLetters)
    stop4 = (timeit.default_timer() - start4) * 1000
    print("Odkodowany ciąg w LZ77: ", decodedString)
    print("Czas dekodowania LZ77: ", stop4)
    
    print("- - - - - - - - - - - - - - - - - -")
    
    start1 = timeit.default_timer()
    [encodedNumbers, encodedLetters, dictionary] = encode_lz78(stringToEncode)
    stop1 = (timeit.default_timer() - start1) * 1000
    print("Zakodowany ciąg w LZ78: ", end="")
    i = 0
    while i < len(encodedNumbers):
        print ("<{0}, {1}>".format(encodedNumbers[i], encodedLetters[i]), end=" ")
        i = i + 1
    print("\nCzas kodowania LZ78: ", stop1)
    print("Odkodowany ciąg w LZ78: ", end = "")
    start2 = timeit.default_timer()
    decode_lz78(encodedNumbers, encodedLetters, dictionary)
    stop2 = (timeit.default_timer() - start2) * 1000
    print("\nCzas dekodowania LZ78: ", stop2)
    
    print("- - - - - - - - - - - - - - - - - -")
    
    start5 = timeit.default_timer()
    [encoded, dictionary] = encode_lzw(stringToEncode)
    stop5 = (timeit.default_timer() - start5) * 1000
    print("Zakodowany ciąg w LZW: ", encoded)
    print("Czas kodowania LZW: ", stop5)
    print("Odkodowany ciąg w LZW: ", end = "")
    start6 = timeit.default_timer()
    decode_lzw(encoded, dictionary)
    stop6 = (timeit.default_timer() - start6) * 1000
    print("\nCzas dekodowania LZW: ", stop6)

Zakodowany ciąg w LZ77: <0, 0, w> <0, 0, s> <0, 0, z> <0, 0, c> <0, 0, z> <0, 0, e> <0, 0, b> <0, 0, r> <4, 2, s> <3, 1, y> <0, 0, n> <0, 0, i> <0, 0, e> <0, 0, c> <0, 0, h> <0, 0, r> <0, 0, z> <0, 0, a> <0, 0, s> <3, 1, c> <0, 0, z> <0, 0, b> <0, 0, r> <3, 1, m> <0, 0, i> <0, 0, w> <0, 0, t> <0, 0, r> <0, 0, z> <0, 0, c> <0, 0, i> <0, 0, n> <0, 0, i> <0, 0, e> 
Czas kodowania LZ77:  0.2681000000848144
Odkodowany ciąg w LZ77:  ['w', 's', 'z', 'c', 'z', 'e', 'b', 'r', 'z', 'e', 's', 'z', 'y', 'n', 'i', 'e', 'c', 'h', 'r', 'z', 'a', 's', 'z', 'c', 'z', 'b', 'r', 'z', 'm', 'i', 'w', 't', 'r', 'z', 'c', 'i', 'n', 'i', 'e']
Czas dekodowania LZ77:  0.05379999993238016
- - - - - - - - - - - - - - - - - -
Zakodowany ciąg w LZ78: <0, w> <0, s> <0, z> <0, c> <3, e> <0, b> <0, r> <5, s> <3, y> <0, n> <0, i> <0, e> <4, h> <7, z> <0, a> <2, z> <4, z> <6, r> <3, m> <11, w> <0, t> <14, c> <11, n> <11, e> 
Czas kodowania LZ78:  0.08909999996831175
Odkodowany ciąg w LZ78: wszczebrzeszyniechrzaszczbrzmi

In [55]:
def longest_common_substring(s1, s2):
    maxLongest = 0
    offset = 0
    for i in range(0, len(s1)):
        longest = 0
        if ((i == len(s1) - len(s2) - 2)):
            break            
        for j in range(0, len(s2)):
            if (i+j < len(s1)):
                if s1[i+j] == s2[j]:
                    longest = longest + 1
                    if (maxLongest < longest):
                        maxLongest = longest
                        offset = i
                else:
                    break
            else:
                break
    return maxLongest, offset



if __name__ == "__main__":
    stringToEncode = "aacaacabcabaaac"
    searchWindowSize = 4
    previewWindowSize = 6
    [encodedNumbers, encodedSizes, encodedLetters] = encode_lz77(stringToEncode, searchWindowSize, previewWindowSize)
    print("Encoded string: ", end="")
    i = 0
    while i < len(encodedNumbers):
        print ("<{0}, {1}, {2}>".format(encodedNumbers[i], encodedSizes[i], encodedLetters[i]), end=" ")
        i = i + 1
    print('\n')
    decodedString = decode_lz77(encodedNumbers, encodedSizes, encodedLetters)
    print("Decoded string: ", decodedString)

Encoded string: <0, 0, a> <0, 0, a> <0, 0, c> <0, 0, a> <0, 0, a> <0, 0, c> <3, 1, b> <3, 3, a> <3, 1, a> <0, 0, c> 

Decoded string:  ['a', 'a', 'c', 'a', 'a', 'c', 'a', 'b', 'c', 'a', 'b', 'a', 'a', 'a', 'c']


In [57]:
def longest_common_substring(s1, s2):
    maxLongest = 0
    offset = 0
    for i in range(0, len(s1)):
        longest = 0
        if ((i == len(s1) - len(s2) - 2)):
            break            
        for j in range(0, len(s2)):
            if (i+j < len(s1)):
                if s1[i+j] == s2[j]:
                    longest = longest + 1
                    if (maxLongest < longest):
                        maxLongest = longest
                        offset = i
                else:
                    break
            else:
                break
    return maxLongest, offset

    
def encode_lzw(text):
    dictionary = dict()
    i = 0
    index = 1
    while i < len(text):
        if text[i] in dictionary:
            i = i + 1
        else:
            dictionary[text[i]] = index
            index = index + 1

    i = 0
    encoded = []
    while i < len(text):
        j = 0
        stringToBeSaved = text[i]
        while stringToBeSaved in dictionary and i+j < len(text):
            indexInDictionary = dictionary[stringToBeSaved]
            length = len(stringToBeSaved)
            if (i+j == len(text) - 1):
                break
            j = j + 1
            stringToBeSaved = stringToBeSaved + text[i+j]
        i = i + length
        encoded.append(indexInDictionary)
        if (stringToBeSaved not in dictionary):
            dictionary[stringToBeSaved] = index
        index = index + 1

    stop = datetime.datetime.now()
    return encoded, dictionary
        
def decode_lzw(encoded, dictionary):
    i = 0
    while i < len(encoded):
        print(list(dictionary.keys())[list(dictionary.values()).index(encoded[i])], end="")
        i = i+1
    print('\n')
    

if __name__ == "__main__":
    start = datetime.datetime.now()
#     stringToEncode = "bbbbbabaaabababba"
    stringToEncode = "aaabbbbbbaaaabbbb"
    [encoded, dictionary] = encode_lzw(stringToEncode)
    stop  = datetime.datetime.now()
    print(start)
    print(stop)
    print("Encoded string: ", encoded)
    print("Decoded string: ", end = "")
    decode_lzw(encoded, dictionary)

2021-06-07 23:31:38.669141
2021-06-07 23:31:38.669141
Encoded string:  [1, 3, 2, 5, 6, 3, 4, 6]
Decoded string: aaabbbbbbaaaabbbb

